Mountig the Google_drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')


In [0]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation,Dropout
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from keras.models import load_model

In [0]:
import tensorflow as tf
tf.compat.v1.get_default_graph
mobile = keras.applications.mobilenet.MobileNet()
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [0]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(512,activation='relu')(x) #dense layer 
preds=Dense(6,activation='softmax')(x) #final layer with softmax activation outupt is six

model=Model(inputs=base_model.input,outputs=preds)  #specify the inputs and output
#

**Now** a model has been created based on our architecture

In [0]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128 

In [0]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,horizontal_flip = True) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/Intel_image/seg_train',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True,)
test_datagen = ImageDataGenerator(rescale = 1./255,)
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/Intel_image/seg_test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical',shuffle = True,)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [0]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
his = model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                    validation_data = test_set,
                   epochs=10)

plt.plot(his.history['loss'], label='train loss')
plt.plot(his.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies
plt.plot(his.history['acc'], label='train acc')
plt.plot(his.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')


Epoch 1/10
438/438 [==============================] - 184s 421ms/step - loss: 0.4114 - acc: 0.8683 - val_loss: 0.7945 - val_acc: 0.7353
Epoch 2/10
438/438 [==============================] - 170s 388ms/step - loss: 0.3057 - acc: 0.8964 - val_loss: 0.5679 - val_acc: 0.7967
Epoch 3/10
438/438 [==============================] - 170s 388ms/step - loss: 0.2664 - acc: 0.9131 - val_loss: 0.5187 - val_acc: 0.8227
Epoch 4/10
438/438 [==============================] - 170s 389ms/step - loss: 0.2402 - acc: 0.9202 - val_loss: 0.4108 - val_acc: 0.8503
Epoch 5/10
438/438 [==============================] - 170s 389ms/step - loss: 0.2286 - acc: 0.9245 - val_loss: 0.4018 - val_acc: 0.8600
Epoch 6/10
438/438 [==============================] - 170s 389ms/step - loss: 0.1975 - acc: 0.9288 - val_loss: 0.5287 - val_acc: 0.8323
Epoch 7/10
438/438 [==============================] - 170s 387ms/step - loss: 0.2024 - acc: 0.9334 - val_loss: 0.4945 - val_acc: 0.8500
Epoch 8/10
438/438 [============================

In [0]:
preprocessed_image = prepare_image('/content/drive/My Drive/Colab Notebooks/Intel_image/seg_test/street/20962.jpg')
predictions = model.predict(preprocessed_image)
dist = {0:"bulding",1:"forest",2:"glacier",3:"mountain",4:"sea",5:"street"}
lis = list(predictions[0])
for i in range(4):
  a = lis.index(max(lis))
  s = str(lis[a]*100)
  print(dist[a],":",s[:6])
  lis[a] = -1

street : 99.978
bulding : 0.0210
sea : 0.0001
glacier : 4.2601


In [0]:
# google colab does not come with torch installed. And also, in course we are using torch 0.4. 
# so following snippet of code installs the relevant version

from os.path import exists
import sys, setuptools, tokenize
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'




In [0]:
import torch
from torchvision import datasets, models, transforms
model_save_name = 'intel_image_classifier.h5'
path = F"/content/drive/My Drive/Colab Notebooks/{model_save_name}" 
torch.save(model, path)